In [6]:
import os
import shutil
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import generate_imgs_for_training

# make directory 
dirname = 'img_data'
if os.path.exists(dirname): shutil.rmtree(dirname) # remove if exists
if not os.path.exists(dirname): os.makedirs(dirname)

pd.read_csv(f"../../datasets/processed/559-train.csv").head(3)

,time,glu_raw,glu,glu_d,glu_t,hr_raw,hr,hr_d,hr_t,iob_raw,iob,hour_raw,hour,basal_raw,bolus_raw,basal,bolus,done
0,2021-12-07 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.054167,-1.852595,0,0.0,0.65,0.0,-1.014677,-0.099243,0
1,2021-12-07 00:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108287,-1.825830,0,0.0,0.65,0.0,-1.014677,-0.099243,0
2,2021-12-07 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.162271,-1.799133,0,0.0,0.65,0.0,-1.014677,-0.099243,0


In [7]:
# set participant id
df_all = pd.DataFrame
for pid in ["559", "563", "570", "575", "588", "591"]:

    # read data
    df = pd.read_csv(f"../../datasets/processed/{pid}-train.csv")
    df[["iob_raw", "basal_raw", "bolus_raw"]] = df[["iob_raw", "basal_raw", "bolus_raw"]].apply(lambda col: col.map(lambda x: 0 if x < 0 else x))
    df = df.dropna(axis=0)

    # discretize basal and bolus
    bins_for_basal_bounds = np.linspace(-0.1, 5.1, 257) # 256 - upper bound is 5
    bins_for_bolus_bounds = np.linspace(-0.1, 25.1, 257) # 256 - upper bound is 25

    df['basal_binned'] = pd.cut(df['basal_raw'], bins=bins_for_basal_bounds, labels=False).astype(int)
    df['bolus_binned'] = pd.cut(df['bolus_raw'], bins=bins_for_bolus_bounds, labels=False).astype(int)

    # generation settings
    step_size = 10 # skip every n points
    plot_interval = 50 # n datapoints before current to show in graph

    # Generate and store plot images
    for i in tqdm(range(plot_interval, len(df), step_size)): 
        df = generate_imgs_for_training(df, i, plot_interval, pid=pid, dirname=dirname)

    # Add df to df_all dataframe
    if df_all.empty: df_all = df
    else: df_all = pd.concat([df_all, df], ignore_index=True)


# save the dataframe
df_all.to_csv("data.csv")

100%|██████████| 962/962 [05:25<00:00,  2.95it/s]  


In [13]:
df_all

,time,glu_raw,glu,glu_d,glu_t,hr_raw,hr,hr_d,hr_t,iob_raw,...,bolus,done,basal_binned,bolus_binned,time_str,glu_url,hr_url,iob_url,basal_url,bolus_url
0,2021-12-07 13:20:00,206.0,0.557703,-0.581471,-1.257560,73.0,-0.038840,0.182794,-1.276998,8.311250,...,-0.099243,0.0,49.0,1.0,13:20:00,./img_data/559_160_glu.png,./img_data/559_160_hr.png,./img_data/559_160_iob.png,./img_data/559_160_basal.png,./img_data/559_160_bolus.png
1,2021-12-07 13:25:00,196.0,0.415985,-1.467792,-1.171252,69.0,-0.292948,-0.364849,0.197282,8.070943,...,-0.099243,0.0,49.0,1.0,13:25:00,NaN,NaN,NaN,NaN,NaN
2,2021-12-07 13:30:00,181.0,0.203408,-2.206392,-1.430175,69.0,-0.292948,0.000246,0.040640,7.828720,...,-0.099243,0.0,49.0,1.0,13:30:00,NaN,NaN,NaN,NaN,NaN
3,2021-12-07 13:35:00,220.0,0.756108,5.770495,-0.345936,74.0,0.024687,0.456615,0.095925,7.585124,...,-0.099243,0.0,49.0,1.0,13:35:00,NaN,NaN,NaN,NaN,NaN
4,2021-12-07 13:40:00,220.0,0.756108,0.009410,0.430833,74.0,0.024687,0.000246,0.169639,7.340697,...,-0.099243,0.0,49.0,1.0,13:40:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63911,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,./img_data/591_9620_glu.png,./img_data/591_9620_hr.png,./img_data/591_9620_iob.png,./img_data/591_9620_basal.png,./img_data/591_9620_bolus.png
63912,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,./img_data/591_9630_glu.png,./img_data/591_9630_hr.png,./img_data/591_9630_iob.png,./img_data/591_9630_basal.png,./img_data/591_9630_bolus.png
63913,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,./img_data/591_9640_glu.png,./img_data/591_9640_hr.png,./img_data/591_9640_iob.png,./img_data/591_9640_basal.png,./img_data/591_9640_bolus.png
63914,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,./img_data/591_9650_glu.png,./img_data/591_9650_hr.png,./img_data/591_9650_iob.png,./img_data/591_9650_basal.png,./img_data/591_9650_bolus.png


In [19]:
# Remove NaNs and save the cleaned dataframe
df_all_clean = df_all.dropna(subset=["glu_raw", "glu_url"])
df_all_clean.to_csv("data_clean.csv")
print(len(df_all_clean))

5185
